#Amazon Review- Tagging Negative Review in Amazon Product Review with RNN Model

#Introduction


### Connecting to Golab Colab

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Importing Libraries

In [ ]:
import functools
import sys
import csv
import pandas as pd
import torch
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchtext
import tqdm
import time


### Dataset

In [ ]:
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
vectors = torchtext.vocab.FastText()

.vector_cache/wiki.en.vec: 6.60GB [00:38, 171MB/s]                            
100%|██████████| 2519370/2519370 [05:21<00:00, 7841.45it/s]


In [ ]:
df_train = "/content/drive/MyDrive/Masters_Thesis/Dataset/encoded_data/encoded_topic_train.csv"
df_test = "/content/drive/MyDrive/Masters_Thesis/Dataset/encoded_data/encoded_topic_test.csv"
target_list = 'encoded_topic'
train_dataloader, valid_dataloader, vocab_size, pad_index, output_dim, vocab = get_data_loaders(train_dataframe= df_train,valid_dataframe=df_test,target_list=target_list)

### Model Implementation

In [ ]:
embedding_dim = 300
hidden_dim = 300
n_layers = 2
bidirectional = True
dropout_rate = 0.5

model = RNN(vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional, dropout_rate, 
             pad_index)
print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,329,802 trainable parameters


In [ ]:
model = apply_weights(model,vocab)

lr = 5e-4

optimizer = optim.Adam(model.parameters(), lr=lr)
loss_function = nn.CrossEntropyLoss()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
loss_function = loss_function.to(device)

In [ ]:
train(total_epoch=20, model=model, train_loader=train_dataloader)

 Epoch  |  Train Loss  | Train Accuracy |  Elapsed 
---------------------------------------------------
   1    |   0.607467   |    0.688821    |   0.12   
   2    |   0.435458   |    0.812585    |   0.08   
   3    |   0.356165   |    0.859465    |   0.08   
   4    |   0.332448   |    0.868134    |   0.10   
   5    |   0.376030   |    0.837929    |   0.08   
   6    |   0.351976   |    0.852123    |   0.10   
   7    |   0.245915   |    0.905833    |   0.08   
   8    |   0.266844   |    0.894958    |   0.08   
   9    |   0.223019   |    0.915453    |   0.10   
  10    |   0.299287   |    0.877063    |   0.08   
  11    |   0.230593   |    0.911724    |   0.11   
  12    |   0.164671   |    0.938939    |   0.08   
  13    |   0.165456   |    0.941012    |   0.09   
  14    |   0.169614   |    0.935836    |   0.10   
  15    |   0.382607   |    0.834327    |   0.08   
  16    |   0.385679   |    0.832657    |   0.10   
  17    |   0.427893   |    0.810818    |   0.09   
  18    |   

In [ ]:
predictions = valid(model, valid_dataloader) 

 Train Loss  | Train Accuracy |  Elapsed 
-----------------------------------------
  0.351447   |    0.858553    |   0.01   


In [ ]:
path = "/content/drive/MyDrive/Masters_Thesis/Models/RNN"
saveModel(model=model, path=path)

All files saved


In [ ]:
saveCSVValidationResult(predictions,path)

### Data Preprocessing & DataLoader

In [ ]:
def tokenize_data(text, tokenizer, max_length):
    tokens = tokenizer(text)[:max_length]
    length = len(tokens)
    return {'tokens': tokens, 'length': length}

def numericalize_data(example, vocab):
    ids = [vocab[token] for token in example['tokens']]
    return {'ids': ids}

def collate(batch, pad_index,target_list):
    batch_ids = [torch.tensor(i['ids']) for i in batch]  # Convert ids list to tensor
    batch_ids = pad_sequence(batch_ids, padding_value=torch.tensor(pad_index), batch_first=True)
    batch_length = [torch.tensor(i['length']) for i in batch]  # Convert length list to tensor
    batch_length = torch.stack(batch_length)
    batch_label = [torch.tensor(i[target_list]) for i in batch]  # Convert reviewText list to tensor
    batch_label = torch.stack(batch_label)
    batch_text = [(i['reviewText']) for i in batch]
    batch = {'ids': batch_ids,
             'length': batch_length,
             target_list: batch_label,
             'text': batch_text}
    return batch

def get_data_loaders(train_dataframe, valid_dataframe,target_list, max_length=256,min_freq = 5, train_batch_size=16, learning_rate=2e-5):
    df_train = pd.read_csv(train_dataframe)
    df_test = pd.read_csv(valid_dataframe)
    df_train = pd.concat([df_train, df_train['reviewText'].apply(tokenize_data, tokenizer=tokenizer, max_length=max_length).apply(pd.Series)], axis=1)
    df_test = pd.concat([df_test, df_test['reviewText'].apply(tokenize_data, tokenizer=tokenizer, max_length=max_length).apply(pd.Series)], axis=1)
    special_tokens = ['<unk>', '<pad>']
    tokens = df_train['tokens'].tolist()
    vocab = torchtext.vocab.build_vocab_from_iterator(tokens, min_freq=min_freq, specials=special_tokens)
    unk_index = vocab['<unk>']
    pad_index = vocab['<pad>']
    vocab.set_default_index(unk_index)
    df_train = pd.concat([df_train, df_train.apply(numericalize_data, vocab=vocab, axis=1).apply(pd.Series)], axis=1)
    df_test = pd.concat([df_test, df_test.apply(numericalize_data, vocab=vocab, axis=1).apply(pd.Series)], axis=1)
    df_train = df_train[['ids', target_list, 'length','reviewText']]
    df_test = df_test[['ids', target_list, 'length','reviewText']]
    vocab_size = len(vocab)
    output_dim = df_train[target_list].nunique()
    train = df_train.apply(lambda row: {
        'ids': row['ids'],
        'length': row['length'],
        target_list: row[target_list],
        'reviewText': row['reviewText']
        }, axis=1).tolist()
    validation = df_test.apply(lambda row: {
        'ids': row['ids'],
        'length': row['length'],
        target_list: row[target_list],
        'reviewText': row['reviewText']
        }, axis=1).tolist()
    batch_size = 64
    collate_fn = functools.partial(collate, pad_index=pad_index, target_list=target_list)
    train_dataloader = DataLoader(train, batch_size=batch_size, collate_fn=collate_fn, shuffle=True)
    valid_dataloader = DataLoader(validation, batch_size=batch_size, collate_fn=collate_fn)

    return train_dataloader, valid_dataloader, vocab_size, pad_index, output_dim, vocab

### RNN Model

In [ ]:
class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional,
                 dropout_rate, pad_index):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_index)
        self.rnn = nn.RNN(embedding_dim, hidden_dim, n_layers, bidirectional=bidirectional,
                            dropout=dropout_rate, batch_first=True)
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout_rate)
        
    def forward(self, ids, length):
        # ids = [batch size, seq len]
        # length = [batch size]
        embedded = self.dropout(self.embedding(ids))
        # embedded = [batch size, seq len, embedding dim]
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, length, batch_first=True, 
                                                            enforce_sorted=False)
        _, hidden = self.rnn(packed_embedded)
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        if self.rnn.bidirectional:
            hidden = self.dropout(torch.cat([hidden[-1], hidden[-2]], dim=-1))
            # hidden = [batch size, hidden dim * 2]
        else:
            hidden = self.dropout(hidden[-1])
            # hidden = [batch size, hidden dim]
        prediction = self.fc(hidden)
        # prediction = [batch size, output dim]
        return prediction

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


def initialize_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_normal_(m.weight)
        nn.init.zeros_(m.bias)
    elif isinstance(m, nn.LSTM):
        for name, param in m.named_parameters():
            if 'bias' in name:
                nn.init.zeros_(param)
            elif 'weight' in name:
                nn.init.orthogonal_(param)

def apply_weights(model,vocab):
  model.apply(initialize_weights)
  pretrained_embedding = vectors.get_vecs_by_tokens(vocab.get_itos())
  model.embedding.weight.data = pretrained_embedding
  return model

### Training & Validation

In [ ]:
def train(total_epoch, model, train_loader):
    print(f"{'Epoch':^7} | {'Train Loss':^12} | {'Train Accuracy':^12} | {'Elapsed':^9}")
    print("-"*51)
    for epoch in range(total_epoch):
      t0_epoch = time.time() 
      tr_loss = 0
      n_correct = 0
      nb_tr_steps = 0
      nb_tr_examples = 0
      model.train()
      for _,data in enumerate(train_loader, 0):
      # for batch in tqdm.tqdm(dataloader, desc='training...', file=sys.stdout):
        input_ids  = data['ids'].to(device)
        length = data['length']
        targets  = data[target_list].to(device)
        # print(input_ids)
        outputs = model(input_ids,length)

        loss = loss_function(outputs, targets)
        tr_loss += loss.item()
        n_correct += get_accuracy(outputs, targets).item()
        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

      time_elapsed = (time.time() - t0_epoch)/60
      epoch_loss = tr_loss/nb_tr_steps
      epoch_accu = n_correct/nb_tr_steps # np.mean(n_correct)
      print(f"{epoch + 1:^7} | {epoch_loss:^12.6f} | {epoch_accu:^14.6f} | {time_elapsed:^9.2f}")

def valid(model, testing_loader):
    print(f"{'Valid Loss':^12} | {'Valid Accuracy':^12} | {'Elapsed':^9}")
    print("-"*41)
    model.eval()
    t0_epoch = time.time() 
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    predictions = []
    with torch.no_grad():
        for _, data in enumerate(testing_loader, 0):
        #   input_ids, targets, text = tuple(t.to(device) for t in data)
          input_ids  = data['ids'].to(device)
          length = data['length']
          targets  = data[target_list].to(device)
          text = data['text']
          outputs = model(input_ids,length)
          

          loss = loss_function(outputs, targets)
          tr_loss += loss.item()
          n_correct += get_accuracy(outputs, targets).item()

          nb_tr_steps += 1
          nb_tr_examples+=targets.size(0)

          #Todo: get text
          for i in range(len(text)):
            predictions.append({
                'text': text[i],
                'predicted': switch_issue(outputs.argmax(dim=-1)[i].item()),
                'target': switch_issue(targets[i].item())
                })

    time_elapsed = (time.time() - t0_epoch)/60                
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = n_correct/nb_tr_steps # np.mean(n_correct)
    print(f"{epoch_loss:^12.6f} | {epoch_accu:^14.6f} | {time_elapsed:^9.2f}")
    return predictions

def switch_issue(issue_type):
    switcher = {
        1: 'Seller Issue',
        0: 'Product Issue'
        }
    return switcher.get(issue_type, "Invalid Issue Type")

def get_accuracy(prediction, label):
    batch_size, _ = prediction.shape
    predicted_classes = prediction.argmax(dim=-1)
    correct_predictions = predicted_classes.eq(label).sum()
    accuracy = correct_predictions / batch_size
    return accuracy


### Save Files

In [ ]:
def saveCSVValidationResult(predictions, path, fileName="/Topic/back_only_01/result.csv"):
    path = path+"/"+fileName
    with open(path, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['Text', 'Predicted', 'Target'])
        for example in predictions:
            writer.writerow([example['text'], example['predicted'], example['target']])

In [ ]:
def saveModel(model, path):
  MODEL_PATH = path+'/Topic/back_only_01/model.pth'
  torch.save(model.state_dict(), MODEL_PATH)  
  print('All files saved')

def loadModel(path):
  MODEL_PATH = path+'/model.pth'
  model.load_state_dict(torch.load(MODEL_PATH))
  return model, tokenizer